## Kindle Review Sentiment Analysis Project

## About Dataset
Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis

### Best Practices
1. Preprocessing and Cleaning
2. Train Test Split
3. Apply BOW,TF-IDF or Word2Vec (Words--->Vectors)
4. Model Training
5. Model Prediction and Performance Analysis

Google Coab Link: https://colab.research.google.com/drive/1sc20ply9cgh1gSb8QcashzygdMBWn3EA?usp=sharing

In [17]:
import pandas as pd
data=pd.read_csv('all_kindle_review.csv')

In [18]:
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [19]:
df=data[['reviewText','rating']]

In [20]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [21]:
df.shape

(12000, 2)

In [22]:
df.isnull().sum()

,0
reviewText,0
rating,0


In [23]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [24]:
df['rating'].value_counts()

,count
rating,
5,3000
4,3000
3,2000
2,2000
1,2000


### Step 01: Preprocessing and Cleaning

In [25]:
# As we are doing Sentiment Analysis and we have to tell that a review is positive or negative
# So we will have in rating only two values for rating less than 3, 0 means bad for rating greater than 3, 1 means good
df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)

/tmp/ipython-input-3024835294.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)


In [26]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [27]:
## 1. Lower all the cases
df['reviewText']=df['reviewText'].str.lower()

/tmp/ipython-input-3052299212.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].str.lower()


In [28]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [29]:
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))


/tmp/ipython-input-3968272687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
/tmp/ipython-input-3968272687.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
/tmp/ipython-input-3968272687.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [31]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [32]:
# Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [34]:
df['reviewText']=df['reviewText'].apply(lambda x: lemmatize_words(x))

/tmp/ipython-input-3266273754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x: lemmatize_words(x))


In [35]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


### Step 02: Train Test Split

In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],test_size=0.20)

In [37]:
X_test

,reviewText
11762,im going keep short say really enjoyed book im...
4883,already know melody youre normally singing gro...
6221,still patient hospital - scheduled open heart ...
8907,meredith grandmother bed breakfast maine inste...
5427,wild heart really good short story fast easy r...
...,...
3224,m bell continues series another winner humor e...
8577,engaged plot subplots left nothing historic co...
3775,personally found good readi found laughing lou...
5928,reading description book felt great promise un...


### Step:03 Words--->Vectors (BOW, TF-IDF, Word2Vec)

#### Bag Of Words (BOW)

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer=CountVectorizer()
X_train_bow=bow_vectorizer.fit_transform(X_train).toarray()
X_test_bow= bow_vectorizer.transform(X_test).toarray()

#### TF-IDF

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer()
X_train_tfidf=tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf= tfidf_vectorizer.transform(X_test).toarray()

### Word2Vec

In [40]:
from gensim.utils import simple_preprocess

In [41]:
words=[]
for sentence in X_train:
    words.append(simple_preprocess(sentence))


In [42]:
words

[['story',
  'make',
  'feel',
  'like',
  'going',
  'finding',
  'mountain',
  'man',
  'tom',
  'alpha',
  'male',
  'nora',
  'possessed',
  'lot',
  'sas',
  'going',
  'around',
  'circle',
  'handsome',
  'man',
  'big',
  'hand',
  'blistered',
  'noras',
  'bottom',
  'quite',
  'often',
  'loved',
  'determined',
  'keep',
  'safe',
  'character',
  'seemed',
  'real',
  'sense',
  'humor',
  'chuckling',
  'throughout',
  'story',
  'must',
  'read'],
 ['really',
  'liked',
  'halle',
  'puma',
  'series',
  'liked',
  'book',
  'herion',
  'seemed',
  'redeem',
  'prove',
  'others',
  'finally',
  'accepted',
  'pride',
  'pack'],
 ['wanted',
  'love',
  'itthere',
  'many',
  'flaw',
  'book',
  'let',
  'address',
  'prominent',
  'oneslength',
  'issue',
  'long',
  'winded',
  'never',
  'really',
  'getting',
  'anywhereya',
  'romance',
  'nothing',
  'sink',
  'teeth',
  'im',
  'asking',
  'naked',
  'hot',
  'sex',
  'scene',
  'page',
  'tween',
  'romance',
  'd

In [43]:
from gensim.models import Word2Vec
word2vecmodel=Word2Vec(words)

In [44]:
word2vecmodel.wv.index_to_key

['book',
 'story',
 'read',
 'one',
 'character',
 'like',
 'good',
 'would',
 'really',
 'love',
 'time',
 'get',
 'author',
 'reading',
 'series',
 'well',
 'much',
 'first',
 'even',
 'didnt',
 'short',
 'know',
 'way',
 'great',
 'make',
 'could',
 'sex',
 'little',
 'dont',
 'two',
 'thing',
 'want',
 'think',
 'plot',
 'also',
 'find',
 'romance',
 'life',
 'end',
 'im',
 'see',
 'go',
 'scene',
 'enjoyed',
 'never',
 'written',
 'woman',
 'take',
 'lot',
 'many',
 'work',
 'year',
 'kindle',
 'thought',
 'going',
 'found',
 'say',
 'interesting',
 'give',
 'bit',
 'novel',
 'writing',
 'loved',
 'another',
 'liked',
 'feel',
 'man',
 'better',
 'got',
 'back',
 'come',
 'hot',
 'though',
 'enough',
 'still',
 'reader',
 'something',
 'made',
 'people',
 'star',
 'part',
 'review',
 'friend',
 'page',
 'free',
 'world',
 'need',
 'wasnt',
 'keep',
 'bad',
 'cant',
 'new',
 'doesnt',
 'relationship',
 'enjoy',
 'together',
 'next',
 'recommend',
 'start',
 'felt',
 'best',
 'put',

In [45]:
import numpy as np
def average_word2vec(doc):
    instances=[word2vecmodel.wv[word] for word in doc if word in word2vecmodel.wv.index_to_key]
    if(len(instances)<=0):
        return np.zeros(word2vecmodel.wv.vector_size,dtype=np.float32)
    return np.mean(instances,axis=0,dtype=np.float32)

In [46]:
X_train_word2vec=[]
for i in range(len(words)):
    X_train_word2vec.append(average_word2vec(words[i]))

In [47]:
len(X_train_word2vec)

9600

In [48]:
words_test=[]
for sentence in X_test:
    words_test.append(simple_preprocess(sentence))


In [52]:
X_test_word2vec=[]
for i in range(len(words_test)):
    X_test_word2vec.append(average_word2vec(words_test[i]))

In [53]:
len(X_test_word2vec)

2400

### Word2Vec Google News Model

In [51]:
import gensim.downloader as api
google_model=api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [56]:
import numpy as np
def average_word2vec_google(doc):
    instances=[google_model[word] for word in doc if word in google_model]
    if(len(instances)<=0):
      return np.zeros(300, dtype=np.float32)
    return np.mean(instances,axis=0,dtype=np.float32)

In [57]:
X_train_word2vec_google=[]
for i in range(len(words)):
    X_train_word2vec_google.append(average_word2vec_google(words[i]))

In [58]:
X_test_word2vec_google=[]
for i in range(len(words_test)):
    X_test_word2vec_google.append(average_word2vec_google(words_test[i]))

### Step: 04 Model Training And Performance Analysis

#### Bag Of Words (BOW)

In [59]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,classification_report
model_bow=GaussianNB()
model_bow.fit(X_train_bow,y_train)

# Prediction
y_pred_bow=model_bow.predict(X_test_bow)
print(f"Accuracy of Model Using BOW: {accuracy_score(y_test,y_pred_bow)}")
print(f"Classification of Model Using BOW: {classification_report(y_test,y_pred_bow)}")

Accuracy of Model Using BOW: 0.57875
Classification of Model Using BOW:               precision    recall  f1-score   support

           0       0.42      0.66      0.51       808
           1       0.76      0.54      0.63      1592

    accuracy                           0.58      2400
   macro avg       0.59      0.60      0.57      2400
weighted avg       0.64      0.58      0.59      2400



#### TF-IDF

In [60]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,classification_report
model_tfidf=GaussianNB()
model_tfidf.fit(X_train_tfidf,y_train)

# Prediction
y_pred_tfidf=model_tfidf.predict(X_test_tfidf)
print(f"Accuracy of Model Using TF-IDF: {accuracy_score(y_test,y_pred_tfidf)}")
print(f"Classification of Model Using TF-IDF: {classification_report(y_test,y_pred_tfidf)}")

Accuracy of Model Using TF-IDF: 0.5791666666666667
Classification of Model Using TF-IDF:               precision    recall  f1-score   support

           0       0.42      0.64      0.51       808
           1       0.75      0.55      0.63      1592

    accuracy                           0.58      2400
   macro avg       0.58      0.59      0.57      2400
weighted avg       0.64      0.58      0.59      2400



#### Word2Vec And Avg Word2Vec

In [61]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,classification_report
model_word2vec=GaussianNB()
model_word2vec.fit(X_train_word2vec,y_train)

# Prediction
y_pred_word2vec=model_word2vec.predict(X_test_word2vec)
print(f"Accuracy of Model Using word2vec: {accuracy_score(y_test,y_pred_word2vec)}")
print(f"Classification of Model Using word2vec: {classification_report(y_test,y_pred_word2vec)}")

Accuracy of Model Using word2vec: 0.7125
Classification of Model Using word2vec:               precision    recall  f1-score   support

           0       0.55      0.74      0.63       808
           1       0.84      0.70      0.76      1592

    accuracy                           0.71      2400
   macro avg       0.70      0.72      0.70      2400
weighted avg       0.74      0.71      0.72      2400



### Word2Vec Google Pretrained Model

In [62]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,classification_report
model_word2vec_google=GaussianNB()
model_word2vec_google.fit(X_train_word2vec_google,y_train)

# Prediction
y_pred_word2vec_google=model_word2vec_google.predict(X_test_word2vec_google)
print(f"Accuracy of Model Using word2vec_google: {accuracy_score(y_test,y_pred_word2vec_google)}")
print(f"Classification of Model Using word2vec_google: {classification_report(y_test,y_pred_word2vec_google)}")

Accuracy of Model Using word2vec_google: 0.7475
Classification of Model Using word2vec_google:               precision    recall  f1-score   support

           0       0.60      0.78      0.68       808
           1       0.87      0.73      0.79      1592

    accuracy                           0.75      2400
   macro avg       0.73      0.76      0.73      2400
weighted avg       0.78      0.75      0.75      2400

